In [1]:
"""
Processes the raw bike trip data to get information on bike dock locations
and when the docks were placed in those locations.

Desired output columns:

id | first | last | name | lat | lon | rides

where
- id is the station's id
- first is the earliest trip date for the station id
- last is the latest trip date for the station id (included in case docks are removed)
- name is the station's name
- lat and lon are the latitude and longitude of the station's location
- rides is a count of the number of rides found in the data -- it is used to remove dummy stations in the data.
    only stations with more than RIDES_COUNT_THRESHOLD are included in output

This script is abstracte to apply to multiple cities.
DON'T FORGET: update the 'CITY' variable

"""
from datetime import datetime
import math
import os

import pandas as pd
from zipfile import ZipFile
# CITY = 'dc'
# CITY = 'boston'
CITY = 'nyc'
# CITY = 'chicago'
# CITY = 'la'
# CITY = 'philly'
print('city', CITY)


RIDES_COUNT_THRESHOLD = 100


def get_filepath(city):
    return '../data/' + city + '-bike/'


def transform_date(date):
    try:
        dt = datetime.strptime(date.split(' ')[0], '%m/%d/%Y')
    except ValueError:
        # this dataset is so frustrating lol
        dt = datetime.strptime(date.split(' ')[0], '%Y-%m-%d')
        
    return dt.strftime('%Y-%m-%d')

def open_zipfile(zipfilename):
    # Because someone dropped some gnarly mac osx files into their zips
    zipfile = ZipFile(zipfilename)
    filenames = [f.filename for f in zipfile.infolist()]
    # Return the first file that can be opened  - not all of them have .csv suffix
    for filename in filenames:
        try:
            df = pd.read_csv(zipfile.open(filename))
            return df
        except:
            print('failed to open filename from zip', zipfilename, ': ', filename)
            pass
    raise Exception('unable to read a csv from zipfile %s' % zipfilename)

def open_zipfile_dc(zipfilename):
## DC bike files from 2012 to 2017 have 4 files for each quarter
## this generator yields each of those files

    zipfile = ZipFile(zipfilename)
    files = [f.filename for f in zipfile.infolist()]

    for filename in files:
        try:
            df = pd.read_csv(zipfile.open(filename))
            yield df
        except:
            continue

def open_zipfile_chicago(zipfilename):

    zipfile = ZipFile(zipfilename)
    files = [f.filename for f in zipfile.infolist()]
    for filename in files:
        if filename[:8] == '__MACOSX' or filename.endswith('txt') or 'Divvy_Stations_2' in filename:
            continue
        if '/' in filename:
            if '/Divvy_Stations' in filename:
                continue
        try:
            print(filename)
            df = pd.read_csv(zipfile.open(filename))
            yield df
        except:
            continue

def open_zipfile_la(zipfilename):
    zipfile = ZipFile(zipfilename)
    filenames = [f.filename for f in zipfile.infolist()]
    # Return the first file that can be opened  - not all of them have .csv suffix
    if len(filenames) == 2:
        filename = filenames[1]
    else:
        filename = filenames[0]


   
    try:
        df = pd.read_csv(zipfile.open(filename))
        return df
    except:
        print('failed to open filename from zip', zipfilename, ': ', filename)
        pass
    raise Exception('unable to read a csv from zipfile %s' % zipfilename)


city nyc


In [2]:
"""
make a dict like 
{"id": {"name": "", "lat": "", "lon": "", "first": "", "last": ""}}
where there is one entry for each id
and where the start time is always the earliest found

and then later transform it into a dict like

{'id': [id1, id2, id3], 'col_2': ['a', 'b', 'c', 'd']}

to then make into a dataframe and save as a CSV
"""

# input file column names for indexing data with
start_station_id = 'startstationid'
start_station_name = 'startstationname'
start_station_latitude = 'startstationlatitude'
start_station_longitude = 'startstationlongitude'
starttime = 'starttime'


    
# output file column names
ID = 'id'
NAME = 'name'
LAT = 'lat'
LON = 'lon'
FIRST = 'first'
LAST = 'last'
RIDES = 'rides'


In [4]:

def preprocess_stations_df(df):
    # Because someone can't make data files with uniform column names
    df.columns = df.columns.str.replace('number', 'id')  # 'Station Number' vs Station ID
    df.columns = df.columns.str.replace('date', 'time')  # 'Start Date' vs 'Start Time'
    
    
    df.columns = map(str.lower, df.columns)
    df.columns = df.columns.str.replace('[\ ]', '')
    
    # transform the dates
    df[starttime] = df[starttime].apply(transform_date)
    if CITY == "boston":
        df = preprocess_boston_stations_df(df)
    return df


# Some of the earlier bostons stations data does not include lat,lon coordinates.
# These files contains the lat,lon coordinates (and other data) for station IDs
hubway_stations_locations_filenames = [
    "Hubway_Stations_as_of_July_2017.csv",
    "previous_Hubway_Stations_as_of_July_2017.csv"
]

def get_hubway_stations_locations_df():
    df = pd.DataFrame()
    filenames = [get_filepath(CITY) + fname for fname in hubway_stations_locations_filenames]
    for filename in filenames:
        new_df = pd.read_csv(filename)    
        hubway_stations_locations_column_names = {
            "Station ID": start_station_id,
            "Latitude": start_station_latitude,
            "Longitude": start_station_longitude,
        }
        # Rename the column names to match the rides data that the locations data will be joined with
        new_df.rename(columns=hubway_stations_locations_column_names, inplace=True)
        df = new_df if df.empty else df.append(new_df)
    df.drop_duplicates(subset=[start_station_id], inplace=True)
    return df

hubway_stations_locations_df = None
if CITY == "boston":
    hubway_stations_locations_df = get_hubway_stations_locations_df()


def preprocess_boston_stations_df(df):
    if start_station_latitude in df.columns:
        return df
    # Otherwise this is one of the datasets that is lacking lat, lon info.
    # Add the lat,lon info
    return hubway_stations_locations_df.merge(df, on=start_station_id)

def choose_chicago_columns(filename):
    if filename == 'Divvy_Trips_2018_Q1.zip' or filename == 'Divvy_Trips_2019_Q2.zip':
        return ('03-rentalstartstationid','03-rentalstartstationname', '', '', '01-rentaldetailslocalstarttime' )
    if filename[:5] == 'Divvy' and filename != 'Divvy_Trips_2020_Q1.zip':
        if filename[:7] == 'Divvy_S' or int(filename[12:16]) < 2017:
            return ('from_station_id', 'from_station_name', '', '', 'starttime')
        return ('from_station_id', 'from_station_name', '', '', 'start_time')
    return ('start_station_id', 'start_station_name', 'start_lat', 'start_lng', 'started_at')



# hubway_stations_locations_df.head()

In [5]:
FILENAMES_TO_IGNORE = hubway_stations_locations_filenames + ['indego-stations-2022-04-01.csv'] + ['stations.csv'] + ['stations.json'] + ['metro-bike-share-stations-2022-04-01.csv']# + [more bad filenames here]


def stations_dict_to_df(stations_dict):
    new_dict = {
        ID: [],
        NAME: [],
        LAT: [],
        LON: [],
        FIRST: [],
        LAST: [],
        RIDES: []
    }
    for station_id, station_dict in stations_dict.items():
        new_dict[ID].append(station_id)
        new_dict[NAME].append(station_dict[NAME])
        new_dict[LAT].append(station_dict[LAT])
        new_dict[LON].append(station_dict[LON])
        new_dict[FIRST].append(station_dict[FIRST])
        new_dict[LAST].append(station_dict[LAST])
        new_dict[RIDES].append(station_dict[RIDES])
    
    return pd.DataFrame.from_dict(new_dict)
    


stations_dict = dict()
needs_lat_lon = set()
directory = get_filepath(CITY)
files_count = 0
for filename in os.listdir(directory):
    
    print(filename)
    if filename in FILENAMES_TO_IGNORE:# or filename[:6] not in ['202102']:
        continue
    if CITY == 'chicago':
        start_station_id, start_station_name, start_station_latitude, start_station_longitude, starttime = choose_chicago_columns(filename)
    elif CITY == 'la' or CITY == 'philly':
        
        if (filename.endswith("-2.zip") and CITY == 'la') or ('2016' in filename or 'Q1_2017' in filename and CITY == 'philly'):
            start_station_id = 'start_station_id'
            start_station_name = 'start_station_id'
        else:
            start_station_id = 'start_station'
            start_station_name = 'start_station' # to fill in with other data table later
        start_station_latitude = 'start_lat'
        start_station_longitude = 'start_lon'
        starttime = 'start_time'
   

    elif filename[:4] in ['2021', '2022'] and filename[:6] not in ['202101'] and CITY in ['boston', 'nyc'] or (filename[:4] in ['2020', '2021', '2022'] and filename[:6] not in ['202001', '202002', '202003']) and CITY == 'dc' :
        start_station_id = 'start_station_id'
        start_station_name = 'start_station_name'
        start_station_latitude = 'start_lat'
        start_station_longitude = 'start_lng'
        starttime = 'started_at'
    elif CITY != 'dc':
        start_station_id = 'startstationid'
        start_station_name = 'startstationname'
        start_station_latitude = 'startstationlatitude'
        start_station_longitude = 'startstationlongitude'
        starttime = 'starttime'
    else:
        start_station_id = 'startstationid'
        start_station_name = 'startstation'
        start_station_latitude = ""
        start_station_longitude = ""
        starttime = "starttime"


    fullfilename = directory + filename
    print(files_count, ': handling file', filename)
    files_count+=1
    
    if filename.endswith(".csv"):
        stations_dfs = [pd.read_csv(fullfilename)]
    elif filename.endswith(".zip") and CITY == 'nyc':
        stations_dfs = [open_zipfile(fullfilename)]
    elif filename.endswith(".zip") and CITY == 'la' or CITY =='philly':
        stations_dfs = [open_zipfile_la(fullfilename)]
    elif filename.endswith(".zip") and CITY == 'dc':
        stations_dfs = [df for df in open_zipfile_dc(fullfilename)]
    elif filename.endswith(".zip") and CITY == 'chicago':
        stations_dfs = [df for df in open_zipfile_chicago(fullfilename)]
    else:
        continue
    
    for stations_df in stations_dfs:

        stations_df = preprocess_stations_df(stations_df)
        
        unique_station_ids = stations_df[start_station_id].unique()
        for station_id in unique_station_ids:
            station_df = stations_df[stations_df[start_station_id] == station_id]
            
            if station_id not in stations_dict:
                try:
                    stations_dict[station_id] = {
                        NAME: station_df[start_station_name].iloc[0], 
                        LAT: station_df[start_station_latitude].iloc[0],
                        LON: station_df[start_station_longitude].iloc[0], 
                        FIRST: station_df[starttime].iloc[0], 
                        LAST: station_df[starttime].iloc[0],
                        RIDES: 0,
                    }
                except Exception as e:
                    
                    if type(e).__name__ == 'KeyError':
                        
                        stations_dict[station_id] = {
                            NAME: station_df[start_station_name].iloc[0], 
                            LAT: 0,
                            LON: 0,
                            FIRST: station_df[starttime].iloc[0], 
                            LAST: station_df[starttime].iloc[0],
                            RIDES: 0,
                        }
                        needs_lat_lon.add(station_id)
                    else:    
                        continue
            if station_id in needs_lat_lon and starttime == 'started_at':
                stations_dict[station_id][LAT] = station_df[start_station_latitude].iloc[0]
                stations_dict[station_id][LON] = station_df[start_station_longitude].iloc[0]
                needs_lat_lon.remove(station_id)
            rides_count = len(station_df.index)
            stations_dict[station_id][RIDES] += rides_count
            station_df = station_df.sort_values(by=[starttime])
            if (station_df[starttime].iloc[0] < stations_dict[station_id][FIRST]):
                stations_dict[station_id][FIRST] = stations_df[starttime].iloc[0]
            if (station_df[starttime].iloc[-1] > stations_dict[station_id][LAST]):
                stations_dict[station_id][LAST] = stations_df[starttime].iloc[-1]

if CITY == 'la':
    '''Adding station names to the data frame for LA'''
    name_df = pd.read_csv(get_filepath(CITY) + '/metro-bike-share-stations-2022-04-01.csv', encoding='cp1252')
    for index, row in name_df.iterrows():
        if row['Station_ID'] in stations_dict:
            stations_dict[row['Station_ID']][NAME] = row['Station_Name']
if CITY == 'philly':
    name_df = pd.read_csv(get_filepath(CITY) + '/indego-stations-2022-04-01.csv')
    for index, row in name_df.iterrows():
        if row['Station_ID'] in stations_dict:
            stations_dict[row['Station_ID']][NAME] = row['Station_Name']


stations_df = stations_dict_to_df(stations_dict)
stations_df.head()

.gitignore
0 : handling file .gitignore
.keep
1 : handling file .keep
201306-citibike-tripdata.zip
2 : handling file 201306-citibike-tripdata.zip


C:\Users\Walter\AppData\Local\Temp\ipykernel_15036\4204538325.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns = df.columns.str.replace('[\ ]', '')


201307-citibike-tripdata.zip
3 : handling file 201307-citibike-tripdata.zip


C:\Users\Walter\AppData\Local\Temp\ipykernel_15036\4204538325.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns = df.columns.str.replace('[\ ]', '')


201308-citibike-tripdata.zip
4 : handling file 201308-citibike-tripdata.zip


...

C:\Users\Walter\AppData\Local\Temp\ipykernel_15036\302342831.py:60: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(zipfile.open(filename))
C:\Users\Walter\AppData\Local\Temp\ipykernel_15036\4204538325.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns = df.columns.str.replace('[\ ]', '')


202204-citibike-tripdata.csv.zip
108 : handling file 202204-citibike-tripdata.csv.zip


C:\Users\Walter\AppData\Local\Temp\ipykernel_15036\302342831.py:60: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(zipfile.open(filename))
C:\Users\Walter\AppData\Local\Temp\ipykernel_15036\4204538325.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns = df.columns.str.replace('[\ ]', '')


stations.csv
stations.json


,id,name,lat,lon,first,last,rides
0,444,Broadway & W 24 St,40.742354,-73.989151,2013-06-01,2017-10-31,285239
1,406,Hicks St & Montague St,40.695128,-73.995951,2013-06-01,2021-01-31,129149
2,475,E 15 St & Irving Pl,40.735243,-73.987586,2013-06-01,2017-02-28,127196
3,2008,Little West St & 1 Pl,40.705693,-74.016777,2013-06-01,2021-01-31,331256
4,485,W 37 St & 5 Ave,40.750380,-73.983390,2013-06-01,2021-01-31,246676


In [6]:
# Transform the stations_df

# Remove dummy stations (there are test stations in the data)
# Remove stations with less than RIDES_COUNT_THRESHOLD rides
bad_stations_df = stations_df[stations_df[RIDES] < RIDES_COUNT_THRESHOLD]
print('removing %d bad stations that each have less than %d rides from stations data' % (bad_stations_df.shape[0], RIDES_COUNT_THRESHOLD))
stations_df = stations_df[stations_df[RIDES] >= RIDES_COUNT_THRESHOLD]
## Remove stations that do not have a latitude and longitude measure
bad_stations_2 = stations_df[stations_df[LAT] == 0]
print(f'removing {bad_stations_2.shape[0]} more stations for not having latitude or longitude')
stations_df = stations_df[stations_df[LAT] != 0]

removing 141 bad stations that each have less than 100 rides from stations data
removing 1 more stations for not having latitude or longitude


In [18]:
bad_stations_df.head(10)

,id,name,lat,lon,first,last,rides


In [7]:
"""For the boston hubway/blue bikes data there will be duplicates because
when management changed from hubway to Bluebikes, the data fromat did too
This includes the station id/numbers and names AND lat/lon!
Task: deduplicate stations

Idea to understand data: sort the stations so the potential duplicates are next to each other
when merging/deduping data make sure to keep the earliest first and the latest last.

approach to deduplicating stations:
- normalize names and add new temporary column with normalized name
- get list of unique normalized names
- for each name:
    make a df for that name, sorted by [first, last]
    update main df to replace entries with that name with:
        first first
        last last
        last name
        rides as sum of rides
    sort main df by [name, first] and drop duplicates (duplicates on normalized name)
    remove normalized name column
"""

import re

NORMALIZED_NAME = 'normalized_name'

def normalized_station_name(name):
    normalized_name = name.lower()
    normalized_name = normalized_name.replace("former",  "").replace(" ", "")
    normalized_name = re.sub(r'[^a-z0-9]','', normalized_name)
    return normalized_name

if CITY == 'boston' or CITY == 'chicago' or CITY == 'dc' or CITY == 'nyc':
    stations_df[NORMALIZED_NAME] = stations_df[NAME].apply(normalized_station_name)
    normalized_names = stations_df[NORMALIZED_NAME]
    print(normalized_names.shape[0], ' names')
    unique_normalized_names = stations_df[NORMALIZED_NAME].unique()
    print(unique_normalized_names.shape[0], ' unique normalized names') #, unique_normalized_names)


    n = 0
    for normalized_name in unique_normalized_names:
        print(n, 'handling name', normalized_name)
        n+=1
        name_df = stations_df[stations_df[NORMALIZED_NAME] == normalized_name]
        name_df.sort_values(by=[FIRST, LAST], inplace=True)
        first = name_df[FIRST].iloc[0]
        last = name_df[LAST].iloc[-1]
        name = name_df[NAME].iloc[-1]
        rides = name_df[RIDES].sum()
        update_condition = (stations_df[NORMALIZED_NAME] == normalized_name)
        stations_df.loc[update_condition, [FIRST, LAST, NAME, RIDES]] = first, last, name, rides

    stations_dropped_duplicates_df = stations_df.drop_duplicates(subset=[NORMALIZED_NAME])
    print('dropped %s rows based on duplicate names' % (int(stations_df.shape[0]) - int(stations_dropped_duplicates_df.shape[0])))
    stations_dropped_duplicates_df.drop(labels=[NORMALIZED_NAME], axis=1, inplace=True)
    
    stations_df = stations_dropped_duplicates_df


4487  names
1737  unique normalized names
0 handling name broadwayw24st
1 handling name hicksstmontaguest
2 handling name e15stirvingpl
3 handling name littlewestst1pl
4 handling name w37st5ave
5 handling name broadwaye14st
6 handling name 9avew22st
7 handling name stantonstchrystiest
8 handling name 9avew14st
9 handling name w54st9ave
10 handling name henrystgrandst
11 handling name dekalbavesportlandave
12 handling name broadwayw29st
13 handling name e33st2ave
14 handling name murraystwestst
15 handling name e20stparkave


C:\Users\Walter\AppData\Local\Temp\ipykernel_15036\805789800.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  name_df.sort_values(by=[FIRST, LAST], inplace=True)
C:\Users\Walter\AppData\Local\Temp\ipykernel_15036\805789800.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  name_df.sort_values(by=[FIRST, LAST], inplace=True)
C:\Users\Walter\AppData\Local\Temp\ipykernel_15036\805789800.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-

16 handling name lispenardstbroadway
17 handling name w26st8ave
18 handling name w4st7aves
19 handling name e58st3ave
20 handling name w53st10ave
21 handling name w22st8ave
22 handling name w13st7ave
23 handling name e31st3ave
24 handling name clevelandplspringst
25 handling name allenststantonst
26 handling name w15st7ave
27 handling name e4st2ave
28 handling name forsythstbroomest
29 handling name christopherstgreenwichst
30 handling name bayardstbaxterst
31 handling name atlanticavefortgreenepl


C:\Users\Walter\AppData\Local\Temp\ipykernel_15036\805789800.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  name_df.sort_values(by=[FIRST, LAST], inplace=True)
C:\Users\Walter\AppData\Local\Temp\ipykernel_15036\805789800.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  name_df.sort_values(by=[FIRST, LAST], inplace=True)
C:\Users\Walter\AppData\Local\Temp\ipykernel_15036\805789800.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-

32 handling name ashlandplhansonpl
33 handling name e10stavenuea
34 handling name w41st8ave
35 handling name lafayetteste8st
36 handling name w45st6ave1
37 handling name rivingtonstchrystiest
38 handling name 10avew28st
39 handling name pershingsquaresouth
40 handling name 6avebroomest
41 handling name nycbsdepotdel
42 handling name harrisonsthudsonst
43 handling name e7stavenuea
44 handling name franklinstwbroadway
45 handling name w52st11ave


...

C:\Users\Walter\AppData\Local\Temp\ipykernel_15036\805789800.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  name_df.sort_values(by=[FIRST, LAST], inplace=True)
C:\Users\Walter\AppData\Local\Temp\ipykernel_15036\805789800.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  name_df.sort_values(by=[FIRST, LAST], inplace=True)
C:\Users\Walter\AppData\Local\Temp\ipykernel_15036\805789800.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-

1688 handling name broadwaytw48st
1689 handling name e92stparkave
1690 handling name riversidedrbroadway
1691 handling name duanesthudsonst
1692 handling name e7stavec
1693 handling name 48ave31st
1694 handling name sterlingpl5ave
1695 handling name e7staveb
1696 handling name 23rd31st
1697 handling name e4staveb
1698 handling name e5stavea
1699 handling name e14st1ave
1700 handling name w12sthudsonst
1701 handling name greenestprincest
1702 handling name e6st2ave
1703 handling name ehoustonstcolumbiast
1704 handling name e10st2ave
1705 handling name southstpikest
1706 handling name w34stthudsonblvde


C:\Users\Walter\AppData\Local\Temp\ipykernel_15036\805789800.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  name_df.sort_values(by=[FIRST, LAST], inplace=True)
C:\Users\Walter\AppData\Local\Temp\ipykernel_15036\805789800.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  name_df.sort_values(by=[FIRST, LAST], inplace=True)
C:\Users\Walter\AppData\Local\Temp\ipykernel_15036\805789800.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-

1707 handling name greenwichstrectorst
1708 handling name hudsonstnmoorest
1709 handling name westbroadwaywattsst
1710 handling name broadwaymorrisst
1711 handling name washingtonstlaightst
1712 handling name rutgerssthenryst
1713 handling name waterstfletcherst
1714 handling name e26st3ave
1715 handling name e20stfdrdr
1716 handling name e22st2ave
1717 handling name 7avesbleeckerst
1718 handling name orchardstgrandst
1719 handling name hudsonblvdew36st
1720 handling name e106st2ave
1721 handling name 6avewalkerst
1722 handling name 50st7ave
1723 handling name kenmarestelizabethst
1724 handling name w30st8ave
1725 handling name aveae14st
1726 handling name aveae11st
1727 handling name kentavedivisionave
1728 handling name e156stcourtlandtave
1729 handling name valentineavee181st
1730 handling name w56st8ave
1731 handling name riverterwarrenst
1732 handling name broadwaye19st
1733 handling name walkerstbaxterst
1734 handling name w35st8ave
1735 handling name w111st5ave
1736 handling nam

C:\Users\Walter\AppData\Local\Temp\ipykernel_15036\805789800.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  name_df.sort_values(by=[FIRST, LAST], inplace=True)
C:\Users\Walter\AppData\Local\Temp\ipykernel_15036\805789800.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  name_df.sort_values(by=[FIRST, LAST], inplace=True)
C:\Users\Walter\AppData\Local\Temp\ipykernel_15036\805789800.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-

In [8]:
# Save the data to CSV
save_to_csvfilename = directory + 'stations.csv'
stations_df.to_csv(save_to_csvfilename)
print('wrote data to ', save_to_csvfilename)

wrote data to  ../data/nyc-bike/stations.csv


In [9]:
# Save the data to JSON that will be used in web app
import json

stations = []
for index, row in stations_df.iterrows():
    # Transform the date
    date = row[5]
    
    stations.append({
        ID: str(row[ID]),
        NAME: str(row[NAME]),
        LAT: row[LAT],
        LON: row[LON],
        FIRST: transform_date(row[FIRST]),
        LAST: transform_date(row[LAST]),
    })

json = json.dumps(stations)

save_to_jsonfilename = directory + 'stations.json'
with open(save_to_jsonfilename, 'w') as f:
    f.write(json)
print("Data written to stations.json")

Data written to stations.json


In [10]:
stations_df.head()

,id,name,lat,lon,first,last,rides
0,444,Broadway & W 24 St,40.742354,-73.989151,2013-06-01,2017-10-31,285239
1,406,Hicks St & Montague St,40.695128,-73.995951,2013-06-01,2022-04-11,156951
2,475,E 15 St & Irving Pl,40.735243,-73.987586,2013-06-01,2017-02-28,127196
3,2008,Little West St & 1 Pl,40.705693,-74.016777,2013-06-01,2022-04-11,395638
4,485,W 37 St & 5 Ave,40.750380,-73.983390,2013-06-01,2022-04-11,293405
